In [1]:
import os
import numpy as np
import pandas as pd
import glob
import csv
import time

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau

import cv2
import mediapipe as mp

In [20]:
TRAIN_DIR = "./output/train_second/"
TEST_DIR = "./output/test/"
IMG_SIZE = 48

In [26]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=3,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 2230 images belonging to 24 classes.
Found 1967 images belonging to 24 classes.


In [15]:
labels = list(train_gen.class_indices.keys())
print(labels)

['B', 'D', 'E']


In [5]:
model2 = Sequential()
model2.add(Conv2D(128, kernel_size=(5,5),
                 strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
model2.add(MaxPool2D(pool_size=(4,4), strides=2, padding='same'))
model2.add(Dropout(0.4))

model2.add(Conv2D(64, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
model2.add(Dropout(0.4))

model2.add(Conv2D(32, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
model2.add(Dropout(0.4))
          
model2.add(Flatten())
model2.add(Dense(units=512,activation='relu'))
model2.add(Dropout(rate=0.25))
model2.add(Dense(units=24,activation='softmax'))
model2.summary()

model2.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 128)       3328      
                                                                 
 max_pooling2d (MaxPooling2  (None, 24, 24, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 24, 24, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 12, 12, 64)        0

In [80]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

model2.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size,
    callbacks = [learning_rate_reduction]
)

Epoch 1/20
225/225 [==============================] - 274s 1s/step - loss: 1.9754 - accuracy: 0.3655 - val_loss: 0.8952 - val_accuracy: 0.7510 - lr: 0.0010
Epoch 2/20
225/225 [==============================] - 142s 632ms/step - loss: 0.7305 - accuracy: 0.7333 - val_loss: 0.6914 - val_accuracy: 0.7937 - lr: 0.0010
Epoch 3/20
225/225 [==============================] - 149s 663ms/step - loss: 0.4406 - accuracy: 0.8420 - val_loss: 0.7170 - val_accuracy: 0.7906 - lr: 0.0010
Epoch 4/20
225/225 [==============================] - 147s 651ms/step - loss: 0.3152 - accuracy: 0.8880 - val_loss: 0.6842 - val_accuracy: 0.8177 - lr: 0.0010
Epoch 5/20
225/225 [==============================] - 154s 682ms/step - loss: 0.2255 - accuracy: 0.9203 - val_loss: 0.6232 - val_accuracy: 0.7969 - lr: 0.0010
Epoch 6/20
225/225 [==============================] - ETA: 0s - loss: 0.1934 - accuracy: 0.9320
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
225/225 [===========================

In [81]:
model2.save("./output/model_all_alpha_85.h5")

# Second Round for Model 2 (85%)

In [27]:
model2 = load_model("./models/model_all_alpha_85.h5")

# Freeze all layers in the model
for layer in model2.layers:
    layer.trainable = False

# Recompile
model2.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

model2.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=test_gen,
    validation_steps=test_gen.samples // batch_size,
    callbacks = [learning_rate_reduction]
)

Epoch 1/20
17/17 [==============================] - 14s 780ms/step - loss: 8.8690 - accuracy: 0.2693 - val_loss: 0.6058 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 2/20
17/17 [==============================] - 11s 661ms/step - loss: 8.8694 - accuracy: 0.2488 - val_loss: 0.5999 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 3/20
17/17 [==============================] - 11s 664ms/step - loss: 8.8936 - accuracy: 0.2697 - val_loss: 0.6042 - val_accuracy: 0.8573 - lr: 0.0010
Epoch 4/20
17/17 [==============================] - 11s 677ms/step - loss: 9.0334 - accuracy: 0.2633 - val_loss: 0.5950 - val_accuracy: 0.8583 - lr: 0.0010
Epoch 5/20
17/17 [==============================] - 11s 663ms/step - loss: 8.9902 - accuracy: 0.2564 - val_loss: 0.6097 - val_accuracy: 0.8562 - lr: 0.0010
Epoch 6/20
17/17 [==============================] - ETA: 0s - loss: 8.8401 - accuracy: 0.2774
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
17/17 [==============================] - 12s 702ms

In [28]:
model2.save("./models/test.h5")

In [ ]:
model2 = Sequential()
model2.add(Conv2D(128, kernel_size=(5,5),
                 strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
model2.add(MaxPool2D(pool_size=(4,4), strides=2, padding='same'))
model2.add(Conv2D(64, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
model2.add(Conv2D(32, kernel_size=(2,2),
                 strides=1, activation='relu', padding='same'))
model2.add(MaxPool2D((4,4), 2, padding='same'))
          
model2.add(Flatten())
model2.add(Dense(units=512,activation='relu'))
model2.add(Dropout(rate=0.25))
model2.add(Dense(units=24,activation='softmax'))
model2.summary()

model2.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(24, activation='softmax'))

In [60]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(24, activation='softmax'))

In [61]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 48, 48, 128)       1280      
                                                                 
 max_pooling2d_42 (MaxPooli  (None, 24, 24, 128)       0         
 ng2D)                                                           
                                                                 
 dropout_60 (Dropout)        (None, 24, 24, 128)       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 22, 22, 256)       295168    
                                                                 
 max_pooling2d_43 (MaxPooli  (None, 11, 11, 256)       0         
 ng2D)                                                           
                                                                 
 dropout_61 (Dropout)        (None, 11, 11, 256)     

In [62]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [63]:
model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_gen.samples // batch_size
)

Epoch 1/20
50/50 [==============================] - 119s 2s/step - loss: 2.9664 - accuracy: 0.0874 - val_loss: 9.0711 - val_accuracy: 0.1042
Epoch 2/20
 3/50 [>.............................] - ETA: 1:36 - loss: 2.7814 - accuracy: 0.0938

KeyboardInterrupt: 

In [36]:
model.save("./output/model.h5")

In [24]:
model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_gen.samples // batch_size
)

Epoch 1/20
18/18 [==============================] - 12s 574ms/step - loss: 1.1925 - accuracy: 0.5618 - val_loss: 0.5878 - val_accuracy: 0.9355
Epoch 2/20
18/18 [==============================] - 10s 540ms/step - loss: 0.2335 - accuracy: 0.9366 - val_loss: 0.0506 - val_accuracy: 0.9863
Epoch 3/20
18/18 [==============================] - 10s 526ms/step - loss: 0.0269 - accuracy: 0.9938 - val_loss: 0.0433 - val_accuracy: 0.9863
Epoch 4/20
18/18 [==============================] - 10s 526ms/step - loss: 0.0123 - accuracy: 0.9965 - val_loss: 0.0506 - val_accuracy: 0.9863
Epoch 5/20
18/18 [==============================] - 10s 527ms/step - loss: 0.0057 - accuracy: 0.9987 - val_loss: 0.0533 - val_accuracy: 0.9883
Epoch 6/20
18/18 [==============================] - 10s 544ms/step - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0643 - val_accuracy: 0.9863
Epoch 7/20
18/18 [==============================] - 10s 521ms/step - loss: 0.0029 - accuracy: 0.9996 - val_loss: 0.0332 - val_accuracy: 0.9902

In [25]:
model.save("./output/model_v2.h5")